In [1]:
# =============================================================================
# =============================================================================
# # 문제 01 유형(DataSet_01.csv 이용)
#
# 구분자 : comma(“,”), 4,572 Rows, 5 Columns, UTF-8 인코딩
#
# 글로벌 전자제품 제조회사에서 효과적인 마케팅 방법을 찾기
# 위해서 채널별 마케팅 예산과 매출금액과의 관계를 분석하고자
# 한다.
# 컬 럼 / 정 의  /   Type
# TV   /     TV 마케팅 예산 (억원)  /   Double
# Radio / 라디오 마케팅 예산 (억원)  /   Double
# Social_Media / 소셜미디어 마케팅 예산 (억원)  / Double
# Influencer / 인플루언서 마케팅
# (인플루언서의 영향력 크기에 따라 Mega / Macro / Micro /
# Nano) / String

# SALES / 매출액 / Double
# =============================================================================
# =============================================================================

import pandas as pd

data1 = pd.read_csv('./Dataset/Dataset_01.csv')

print(data1.info())
print()
print(data1.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TV            4562 non-null   float64
 1   Radio         4568 non-null   float64
 2   Social_Media  4566 non-null   float64
 3   Influencer    4572 non-null   object 
 4   Sales         4566 non-null   float64
dtypes: float64(4), object(1)
memory usage: 178.7+ KB
None

Index(['TV', 'Radio', 'Social_Media', 'Influencer', 'Sales'], dtype='object')


In [2]:
# =============================================================================
# 1. 데이터 세트 내에 총 결측값의 개수는 몇 개인가? (답안 예시) 23
# =============================================================================

data1.isna().sum().sum()

# 답 : 26

26

In [3]:
# [참고] : 결측치가 포함된 행의 수
data1.isna().any(axis=1).sum()

26

In [4]:
# ============================================================================
# 2. TV, Radio, Social Media 등 세 가지 다른 마케팅 채널의 예산과 매출액과의 상관분석을
# 통하여 각 채널이 매출에 어느 정도 연관이 있는지 알아보고자 한다.
# - 매출액과 가장 강한 상관관계를 가지고 있는 채널의 상관계수를 소수점 5번째
# 자리에서 반올림하여 소수점 넷째 자리까지 기술하시오. (답안 예시) 0.1234
# =============================================================================

var_list = ['TV','Radio','Social_Media','Sales']

q1_2 = data1.corr().drop('Sales')['Sales'].abs()

# 상관계수 소수점 5번째자리에서 반올림하여 소수점 넷째 자리까지 기술
print(round(q1_2.max(), 4))

#  답 : 0.9995

0.9995


In [5]:
print(q1_2.max()) # 최대값
print(q1_2.idxmax())  # 인덱스명
print(q1_2.argmax())  # 위치번호
print(q1_2.nlargest(1))  # 결과 인덱스명

0.999497444941335
TV
0
TV    0.999497
Name: Sales, dtype: float64


In [7]:
# =============================================================================
# 3. 매출액을 종속변수, TV, Radio, Social Media의 예산을 독립변수로 하여 회귀분석을
# 수행하였을 때, 세 개의 독립변수의 회귀계수를 큰 것에서부터 작은 것 순으로
# 기술하시오.
# - 분석 시 결측치가 포함된 행은 제거한 후 진행하며, 회귀계수는 소수점 넷째 자리
# 이하는 버리고 소수점 셋째 자리까지 기술하시오. (답안 예시) 0.123
# =============================================================================

var_list = ['TV','Radio','Social_Media']

# 결측치가 포함된 행 제거
q1_3 = data1.dropna()

# 회귀계수
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from statsmodels.api import OLS, add_constant

# LinearRegression
lm = LinearRegression(fit_intercept=True)
lm.fit(q1_3[var_list], q1_3['Sales'])

#dir(lm)

print(lm.intercept_)  # 상수항/절편, -0.1339630519422883
print()
print(lm.coef_)  # 회귀계수. [ 3.56256963 -0.00397039  0.00496402]

# 답 : 3.562, 0.004, -0.003

-0.1339630519422883

[ 3.56256963 -0.00397039  0.00496402]


### scikit-learn 패키지 사용한 선형 회귀분석
1. `LinearRegression` 클래스 객체 생성
    ```
    model = LinearRegression(fit_intercept=True)
    ```
    `fit_intercept` 인수는 모형에 상수항이 있는가 없는가를 결정하는 인수 (있으면 True)
    
2. `fit` 메서드로 모형 추정. 상수항 결합을 자동으로 해주므로 사용자가 직접 `add_constant` 등의
    명령을 써서 상수항 결합을 할 필요는 없다.
    ```
    model = model.fit(X,y)
    ```
    회귀분석을 하고 나면 모형 객체는 다음과 같은 속성을 가지게 됨
    - `coef_` : 추정된 가중치 벡터
    - `intercept_` : 추정된 상수항
    
3. `predict` 메서드로 새로운 입력 데이터에 대한 출력 데이터 예측
    ```
    y_new = model.predict(x_new)
    ```

In [8]:
# 식 제작 : 'Sales~TV+Radio+Social_Media'
form1 = 'Sales~' + '+'.join(var_list)
print(form1)

# ols
q1_3_out = ols(form1, q1_3).fit()

#print(dir(q1_3_out))

q1_3_out.summary()

Sales~TV+Radio+Social_Media


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.505e+06
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        23:59:26   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.274e+04
Df Residuals:                    4542   BIC:                         2.277e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1340      0.103     -1.303      0.193      -0.336       0.068
TV               3.5626      0.003   1051.118      0.000       3.556       3.569
Radio           -0.0040      0.010     -0.406      0.685      -0.023       0.015
Social_Media     0.0050      0.025      0.199      0.842      -0.044       0.054
==============================================================================
Omnibus:                        0.056   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.034
Skew:                          -0.001   Prob(JB):                        0.983
Kurtosis:                       3.013   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
q1_3_out.params.drop('Intercept').sort_values(ascending=False).values

# 답 : [ 3.56256963,  0.00496402, -0.00397039]

array([ 3.56256963,  0.00496402, -0.00397039])

In [10]:
# [참고] 유의미한 변수 찾기
q1_3_out.pvalues[q1_3_out.pvalues < 0.05].index

# TV

Index(['TV'], dtype='object')

In [11]:
# 이상치 확인
outlier_score=q1_3_out.outlier_test()
q1_3[outlier_score['bonf(p)'] < 0.05]  # 이상치 제외 데이터 획득

,TV,Radio,Social_Media,Influencer,Sales


In [12]:
q1_3_out2=OLS(q1_3['Sales'], add_constant(q1_3[var_list])).fit()
q1_3_out2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.505e+06
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        23:59:31   Log-Likelihood:                -11366.
No. Observations:                4546   AIC:                         2.274e+04
Df Residuals:                    4542   BIC:                         2.277e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1340      0.103     -1.303      0.193      -0.336       0.068
TV               3.5626      0.003   1051.118      0.000       3.556       3.569
Radio           -0.0040      0.010     -0.406      0.685      -0.023       0.015
Social_Media     0.0050      0.025      0.199      0.842      -0.044       0.054
==============================================================================
Omnibus:                        0.056   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.034
Skew:                          -0.001   Prob(JB):                        0.983
Kurtosis:                       3.013   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### statsmodels 패키지를 사용한 선형 회귀 분석

1. 독립변수와 종속변수가 모두 포함된 데이터프레임 생성. 상수항 결함은 하지 않아도 됨

2. OLS 클래스 객체 생성
   ```
   model = OLS.from_formul(formul,data=df)
   ```
   또는
   독립변수만 있는 데이터프레임 dfs와 종속변수만 있는 데이터프레임 dfy를 인수로 넣어서 만듦
   이 때는 독립변수만 있는 데이터프레임 dfx가 상수항을 가지고 있어야 함
   ```
   model = OLS(dfy, dfx)
   ```
3. `fit` 메서드로 모형 추정. scikit-learn 패키지와 달리 추정 결과는 
    별도의 RegressionResults 클래스 객체로 출력
    ```
    result = model.fit()
    변수 = ols(식, 데이터).fit()
    ```
    식 : 'y~x1+c(x2)+x3-1'
    - -1의 의미 : 상수항 미포함
    - C() : 범주형 선언 -> 더미변수로 자동 변환 생성
    - 결측치는 자동 제외
    - 범주형 변수는 자동으로 더미변수 생성
    - 강제로 생성하고자 할 경우 변수 앞에 C() 사용
    
4. `RegressionResults` 클래스 객체는 결과 리포트용 `summary()`메서드와 예측을 위한 `prediction`메서드를 제공
    ```
    result.summary()
    y_new = result.predict(x_new)
    ```
    이 때, 예측을 위한 데이터는 추정시와 동일하게 상수항 결합을 해 주어야 함

In [13]:
form2 = 'Sales~' + '+'.join(data1.columns.drop('Sales'))
# form2
# 'Sales~TV+Radio+Social_Media+Influencer'

lm3 = ols(form2, q1_3).fit()

lm3.summary()

lm3.predict(q1_3.drop(columns=['Sales']))

0        56.896653
1        46.196295
2       145.924369
3       295.514743
4        53.251588
           ...    
4567     92.452216
4568    252.713557
4569    156.538029
4570    252.778919
4571    149.427658
Length: 4546, dtype: float64

In [14]:
# anova
anova1 = ols('Sales~Influencer', q1_3).fit()

from statsmodels.stats.anova import anova_lm

anova_lm(anova1)

,df,sum_sq,mean_sq,F,PR(>F)
Influencer,3.0,2.081064e+04,6936.879515,0.801596,0.492813
Residual,4542.0,3.930570e+07,8653.830120,NaN,NaN


In [15]:
# 다중비교, 사후분석
from statsmodels.stats.multicomp import pairwise_tukeyhsd

print(pairwise_tukeyhsd(q1_3['Sales'], q1_3['Influencer']))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
 Macro   Mega  -5.6532 0.4709 -15.7039  4.3974  False
 Macro  Micro  -4.4878 0.6606  -14.547  5.5715  False
 Macro   Nano  -4.3573 0.6835 -14.4471  5.7324  False
  Mega  Micro   1.1655 0.9906  -8.8047 11.1356  False
  Mega   Nano   1.2959 0.9873   -8.705 11.2968  False
 Micro   Nano   0.1305    1.0  -9.8791   10.14  False
-----------------------------------------------------


In [16]:
# [참고]
print(type(q1_3['TV'].values))
print(q1_3['TV'].values.reshape(-1,1).ndim)
print(q1_3[['TV']].ndim)
print(q1_3['TV'].ndim)
# nparray.ndim : 배열의 차원 수 or 배열의 축 수

<class 'numpy.ndarray'>
2
2
1
